In [5]:
'''
sevice elasticsearch start
'''

/bin/bash: sevice: command not found


In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '45447f0361d8',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '4IrRB9I_QtSc9ZaomU5o5g',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [274]:
'''
curl -XGET localhost:9200/_cat/indices
'''

'\ncurl -XGET localhost:9200/_cat/indices\n'

# Wiki 파일 읽어오기 & 전처리

In [2]:
path = '/opt/ml/data/wikipedia_documents.json'

In [25]:
import json
with open(path, 'r', encoding="utf-8") as f:
    wiki = json.load(f)
wiki['42']

{'text': '1971년 정주영 회장은 혼자서 미포만 해변 사진 한 장과 외국 조선소에서 빌린 유조선 설계도 하나 들고 유럽을 돌았다. 차관을 받기 위해서였다. 부정적인 반응만 받다가 1971년 9월 영국 바클레이 은행의 차관을 받기 위한 추천서를 부탁하기 위해 A&P 애플도어의 롱바톰 회장을 만났지만 대답은 역시 \'No\'였다. 이 때 정주영은 우리 나라 5백원짜리 지폐를 꺼내 거기 그려진 거북선 그림을 보여줬다. "우리는 영국보다 300년이나 앞선 1500년대에 이미 철갑선을 만들어 외국을 물리쳤소. 비록 쇄국정책으로 시기가 좀 늦어졌지만, 그 잠재력만큼은 충분하다고 생각하오."라며 설득해 결국 차관 도입에 성공할 수 있었다(이 부분은 너무 정주영씨를 미화한 것 같습니다. 사실로 믿고 싶을 만큼 멋진 내용입니다만 5백원짜리 지폐는 1973년 처음으로 발행되었기에 1971년 9월 차관과는 시계열상으로도 맞지 않습니다). 1977년 서울 압구정동 현대아파트의 분양특혜사건으로 재판을 받았으나 무죄로 풀려났다. 건축법 위반에 대해 징역 6월 벌금 500만원에 선고유예 판결을 받았으나 현대산업개발 사장이었던 차남 정몽구가 서울지검 특수부에 구속되어 아들이 아버지 대신 처벌받는 전례가 만들어졌다. 1978년에는 아산사회복지사업재단을 설립하였으며 같은 해 4월 29일 서울 강남구 압구정동에 위치한 현대고등학교를 설립하고 초대이사장으로 취임하였다. 1983년에는 현대전자주식회사를 설립하였다.',
 'title': '정주영'}

# User_Dict_Rules 구성하기

In [4]:
titles = []
for doc in wiki:
    titles.append(wiki[doc]['title'])
titles = list(set(titles))

In [5]:
import re

for i, word in enumerate(titles):

    word_list = []
    word_list.append(re.sub('\s', '', word))
    if len(word.split()) >= 2:
        word_list += re.sub('[()]', '', word).split()

    titles[i] = ' '.join(word_list)

titles = list(set(titles))
if '' in titles:
    titles.pop(titles.index(''))

In [6]:
titles[:20]

['옴니웹',
 '궁수자리VX 궁수자리 VX',
 '수호이Su-33 수호이 Su-33',
 'LiveattheLondonPalladium Live at the London Palladium',
 'NHN다이퀘스트',
 '롯카쿠요시하루 롯카쿠 요시하루',
 '포틀래치(소프트웨어) 포틀래치 소프트웨어',
 '마차시1세 마차시 1세',
 '오이디푸스왕',
 '필립스탁 필립 스탁',
 '압둘카디르칸 압둘 카디르 칸',
 '교초',
 'ARToolKit',
 '김장수',
 '양재익',
 '완월동',
 '개고기',
 '하얏트리젠시홍콩 하얏트 리젠시 홍콩',
 '마더우구',
 '힐튼하노이오페라 힐튼 하노이 오페라']

In [34]:
INDEX_NAME = 'wiki'
index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": { 
                        "type": "stop",
                        "stopwords_path": "stopkr.txt" # stopkr.txt 파일이 /etc/elasticsearch 에 있어야 함
                    }
                },
                "tokenizer": {
                    "my_nori_tokenizer": {
                        "type": "nori_tokenizer", # 노리 형태소 깔아야대는데 에러나면 맨위에 참고해서 깔기
                        "decompound_mode": "mixed",
                        "user_dictionary_rules": titles
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "my_nori_tokenizer", # 위에서 정의한 my_nori_tokenizer
                        "filter" : ["my_stop_filter"] # 위에서 정의한 stopword
                    }
                }
            }
        },
        "mappings": {
            # "dynamic": "strict",
            "properties": {
                "title":{"type": "text", "analyzer": "nori_analyzer"},
                "text": {"type": "text", "analyzer": "nori_analyzer"},
                "document_id": {"type": "long"}
            }
        }
    }

In [35]:
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=index_config)

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki'}

In [36]:
import time
from tqdm.auto import tqdm

for doc_id, doc in tqdm(wiki.items(), total=len(wiki)):
    for key in doc.keys():
        doc[key] = re.sub('~', '\-', doc[key]) # 쿼리에 ~가있어서 안댐
        doc[key] = re.sub('/', '\/', doc[key])
    
    doc['text'] = ''.join(['[', re.sub('\s', '', doc['title']), '] ', doc['text']])

    es.index(index=INDEX_NAME, id=doc_id, body=doc)

  0%|          | 0/60613 [00:00<?, ?it/s]

In [37]:
from pprint import pprint

doc = es.get(index=INDEX_NAME, id=42)
pprint(doc)

{'_id': '42',
 '_index': 'wiki',
 '_primary_term': 1,
 '_seq_no': 42,
 '_source': {'text': '[정주영] [정주영] 1971년 정주영 회장은 혼자서 미포만 해변 사진 한 장과 외국 조선소에서 빌린 '
                     '유조선 설계도 하나 들고 유럽을 돌았다. 차관을 받기 위해서였다. 부정적인 반응만 받다가 1971년 '
                     '9월 영국 바클레이 은행의 차관을 받기 위한 추천서를 부탁하기 위해 A&P 애플도어의 롱바톰 회장을 '
                     "만났지만 대답은 역시 'No'였다. 이 때 정주영은 우리 나라 5백원짜리 지폐를 꺼내 거기 그려진 "
                     '거북선 그림을 보여줬다. "우리는 영국보다 300년이나 앞선 1500년대에 이미 철갑선을 만들어 '
                     '외국을 물리쳤소. 비록 쇄국정책으로 시기가 좀 늦어졌지만, 그 잠재력만큼은 충분하다고 생각하오."라며 '
                     '설득해 결국 차관 도입에 성공할 수 있었다(이 부분은 너무 정주영씨를 미화한 것 같습니다. 사실로 '
                     '믿고 싶을 만큼 멋진 내용입니다만 5백원짜리 지폐는 1973년 처음으로 발행되었기에 1971년 9월 '
                     '차관과는 시계열상으로도 맞지 않습니다). 1977년 서울 압구정동 현대아파트의 분양특혜사건으로 재판을 '
                     '받았으나 무죄로 풀려났다. 건축법 위반에 대해 징역 6월 벌금 500만원에 선고유예 판결을 받았으나 '
                     '현대산업개발 사장이었던 차남 정몽구가 서울지검 특수부에 구속되어 아들이 아버지 대신 처벌받는 전례가 '
                     '만들어졌다. 1978년에는 아산사회복지사업재단을 설립

In [38]:
query = "11-12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?"
# ~는 -으로, /는 없애버림
res = es.search(index=INDEX_NAME, q=query, size=5)
pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '40876',
                    '_index': 'wiki',
                    '_score': 36.021618,
                    '_source': {'text': '[창원성흥사목조아미타여래삼존좌상] [창원 성흥사 '
                                        '목조아미타여래삼존좌상] 창원 성흥사 목조아미타여래삼존좌상은 성흥사 '
                                        '대웅전에 봉안되어 있는 본존불상과 좌우협시보살상이다. 삼존불은 '
                                        '연화대좌 위에 앉아있는데 아미타여래좌상을 중심으로 좌우에 연화가지를 '
                                        '든 관음보살상과 대세지보살상이다 창원 성흥사 목조아미타여래삼존좌상의 '
                                        '얼굴표현과 옷자락, 옷주름 등은 17세기 중엽에 경상도 지역을 '
                                        '중심으로 활동하였던 조각승 희장이 제작한 불상과 매우 유사하다 현재 '
                                        '복장 발원문은 남아있지 않지만 양식적인 특징에서 분석할 수 있는 '
                                        '제작시기와 상통한 점에서 1658년경 희장계 작품일 가능성이 매우 '
                                        '높다. 희장은 양산 통도사와 부산 범어사를 비롯한 경상도와 전라도 '
                           

In [39]:
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))# 문서를 참고해서 score

Doc ID: '40876'  Score: 36.02
Doc ID: '48624'  Score: 35.08
Doc ID: '37102'  Score: 31.65
Doc ID: '37096'  Score: 31.63
Doc ID: '37103'  Score: 31.60


In [40]:
from datasets import load_from_disk
import pandas as pd

train_path = "/opt/ml/data/train_dataset"
train_data = load_from_disk(train_path)

train = train_data['train']
valid = train_data['validation']

In [41]:
for i in range(len(train)):
    print(train[i])
    break

{'__index_level_0__': 42, 'answers': {'answer_start': [229], 'text': ['하원']}, 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05', 'document_id': 18293, 'id': 'mrc-1-000067', 'questi

In [42]:
def es_retrieval(size=5):
    k = 0
    total_len = 0
    past_query = ''
    for i in tqdm(range(len(train))):
        query = train[i]['question']
        answers = train[i]['answers']['text']

        if query == past_query:
            continue
        else:
            total_len += 1
        past_query = query
        
        # answers = re.sub(r"[!/\\~\-\.\+“”%°〈〉<>＜＞・「」;*≪≫·《》‘’『』\"\'_]", '', answers)
        query = re.sub('~', '-', query) # 쿼리에 ~가있어서 안댐
        query = re.sub('/', '|', query)
        print(query)
        res = es.search(index=INDEX_NAME, q=query, size=size)

        for hit in res['hits']['hits']:
            res_text = hit['_source']['text']
            found = False
            for answer in answers:
                if answer in res_text:
                    found = True
                    break
            if found == True:
                k += 1
                break


    return k, total_len

In [45]:
from elasticsearch import RequestError
from pprint import pprint

for size in [1, 5, 10, 15]:
    k, tot = 0, 0
    past_query = ''
    for i in tqdm(range(len(train))):
        query = train[i]['question']
        answers = train[i]['answers']['text']

        if query == past_query:
            continue
        else:
            tot += 1
        past_query = query
        
        # answers = re.sub(r"[!/\\~\-\.\+“”%°〈〉<>＜＞・「」;*≪≫·《》‘’『』\"\'_]", '', answers)
        query = re.sub('~', '-', query) # 쿼리에 ~가있어서 안댐
        query = re.sub('/', '|', query)
        # print(i, query, end='...  ')

        try:
            res = es.search(index=INDEX_NAME, q=query, size=size)
        except RequestError as e:
            # pprint(e.info)
            # break
            print(i, query)
            continue

        # print('search done')
        
        for hit in res['hits']['hits']:
            res_text = hit['_source']['text']
            found = False
            for answer in answers:
                if answer in res_text:
                    found = True
                    break
            if found == True:
                k += 1
                break

    print(f'size {size}: probability {k/tot}')

  0%|          | 0/3952 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


55 경기를 홍보하기 위해 언급된 사람은?
71 러시아를 근대화하려는 노력을 통해 근대 열강의 하나로 자리잡게 한 왕은?
97 김대중을 구하기 위해 캐스퍼 와인버거 미 국방부 장관과 그레그 전 대사를 한국에 파견한 인물은?
102 이익은 그대로 챙기면서 소비자에게 부담을 주는 방식을 시도해 코즈마케팅에 실패한 기업은?
104 트윈 엘리베이터가 승강기 간의 충돌사고를 방지하기 위해 도입한 기술은 무엇인가?
109 맥스가 본사로 복귀하기 위해 활용한 인물은?
114 신비적 체험에 대해 '삶의 원천에서 깨끗하고 맑은 물을 마신 것 같다' 고 표현한 사람은?
116 메가비조스가 도시를 빼앗고, 나중에 생길 반란을 피하기 위해 부순 것은?
122 매년 해수온이 올라가는 현상으로 인해, 자리돔의 서식지가 어디까지 확장되었는가?
125 나치 독일이 지식인이나 귀족 등 65000여 명의 폴란드인을 잡아들이기 위해 준비한 것은?
127 치킨 누들 한 캔을 생산하기 위해 참여하는 회사의 수는?
131 연병환이 영국인 친구의 소개를 통해 구한 중국의 직장은 어디인가?
133 인간의 의식 중, 사이코사이버네틱스를 통해 통제할 수 있는 것은?
158 수령자가 매매계약을 파기하기 위해서 지불해야하는 것은?
172 고려시대 때 마을의 걱정을 없애기 위해 만든 석불은 무엇으로 만들어졌나요?
177 정치하는 엄마들이 피해자들이 정상적인 일상생활을 할 수 있도록 돕기 위해 진행했던 서비스는?
190 중국은 무엇을 통해 리덩후이의 총통 선거를 방해하려 했는가?
204 헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?
219 일반고 슬럼화 문제를 해결하기 위해 교육청에서 내세운 정책 슬로건은?
236 금고를 걸어두기 위해 만들어진 것은 무슨 모양인가요?
237 카스파는 어떤 역할을 담당해 베토밴을 도와주었나?
251 무기를 개편하기 위해 플레이어가 가지고 있어야 하는 것은?
265 바실리 박이 안전을 위해 만나러 간 사람은?
266 영토를 넓히기 위해 대외정

  0%|          | 0/3952 [00:00<?, ?it/s]

55 경기를 홍보하기 위해 언급된 사람은?
71 러시아를 근대화하려는 노력을 통해 근대 열강의 하나로 자리잡게 한 왕은?
97 김대중을 구하기 위해 캐스퍼 와인버거 미 국방부 장관과 그레그 전 대사를 한국에 파견한 인물은?
102 이익은 그대로 챙기면서 소비자에게 부담을 주는 방식을 시도해 코즈마케팅에 실패한 기업은?
104 트윈 엘리베이터가 승강기 간의 충돌사고를 방지하기 위해 도입한 기술은 무엇인가?
109 맥스가 본사로 복귀하기 위해 활용한 인물은?
114 신비적 체험에 대해 '삶의 원천에서 깨끗하고 맑은 물을 마신 것 같다' 고 표현한 사람은?
116 메가비조스가 도시를 빼앗고, 나중에 생길 반란을 피하기 위해 부순 것은?
122 매년 해수온이 올라가는 현상으로 인해, 자리돔의 서식지가 어디까지 확장되었는가?
125 나치 독일이 지식인이나 귀족 등 65000여 명의 폴란드인을 잡아들이기 위해 준비한 것은?
127 치킨 누들 한 캔을 생산하기 위해 참여하는 회사의 수는?
131 연병환이 영국인 친구의 소개를 통해 구한 중국의 직장은 어디인가?
133 인간의 의식 중, 사이코사이버네틱스를 통해 통제할 수 있는 것은?
158 수령자가 매매계약을 파기하기 위해서 지불해야하는 것은?
172 고려시대 때 마을의 걱정을 없애기 위해 만든 석불은 무엇으로 만들어졌나요?
177 정치하는 엄마들이 피해자들이 정상적인 일상생활을 할 수 있도록 돕기 위해 진행했던 서비스는?
190 중국은 무엇을 통해 리덩후이의 총통 선거를 방해하려 했는가?
204 헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?
219 일반고 슬럼화 문제를 해결하기 위해 교육청에서 내세운 정책 슬로건은?
236 금고를 걸어두기 위해 만들어진 것은 무슨 모양인가요?
237 카스파는 어떤 역할을 담당해 베토밴을 도와주었나?
251 무기를 개편하기 위해 플레이어가 가지고 있어야 하는 것은?
265 바실리 박이 안전을 위해 만나러 간 사람은?
266 영토를 넓히기 위해 대외정

  0%|          | 0/3952 [00:00<?, ?it/s]

55 경기를 홍보하기 위해 언급된 사람은?
71 러시아를 근대화하려는 노력을 통해 근대 열강의 하나로 자리잡게 한 왕은?
97 김대중을 구하기 위해 캐스퍼 와인버거 미 국방부 장관과 그레그 전 대사를 한국에 파견한 인물은?
102 이익은 그대로 챙기면서 소비자에게 부담을 주는 방식을 시도해 코즈마케팅에 실패한 기업은?
104 트윈 엘리베이터가 승강기 간의 충돌사고를 방지하기 위해 도입한 기술은 무엇인가?
109 맥스가 본사로 복귀하기 위해 활용한 인물은?
114 신비적 체험에 대해 '삶의 원천에서 깨끗하고 맑은 물을 마신 것 같다' 고 표현한 사람은?
116 메가비조스가 도시를 빼앗고, 나중에 생길 반란을 피하기 위해 부순 것은?
122 매년 해수온이 올라가는 현상으로 인해, 자리돔의 서식지가 어디까지 확장되었는가?
125 나치 독일이 지식인이나 귀족 등 65000여 명의 폴란드인을 잡아들이기 위해 준비한 것은?
127 치킨 누들 한 캔을 생산하기 위해 참여하는 회사의 수는?
131 연병환이 영국인 친구의 소개를 통해 구한 중국의 직장은 어디인가?
133 인간의 의식 중, 사이코사이버네틱스를 통해 통제할 수 있는 것은?
158 수령자가 매매계약을 파기하기 위해서 지불해야하는 것은?
172 고려시대 때 마을의 걱정을 없애기 위해 만든 석불은 무엇으로 만들어졌나요?
177 정치하는 엄마들이 피해자들이 정상적인 일상생활을 할 수 있도록 돕기 위해 진행했던 서비스는?
190 중국은 무엇을 통해 리덩후이의 총통 선거를 방해하려 했는가?
204 헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?
219 일반고 슬럼화 문제를 해결하기 위해 교육청에서 내세운 정책 슬로건은?
236 금고를 걸어두기 위해 만들어진 것은 무슨 모양인가요?
237 카스파는 어떤 역할을 담당해 베토밴을 도와주었나?
251 무기를 개편하기 위해 플레이어가 가지고 있어야 하는 것은?
265 바실리 박이 안전을 위해 만나러 간 사람은?
266 영토를 넓히기 위해 대외정

  0%|          | 0/3952 [00:00<?, ?it/s]

55 경기를 홍보하기 위해 언급된 사람은?
71 러시아를 근대화하려는 노력을 통해 근대 열강의 하나로 자리잡게 한 왕은?
97 김대중을 구하기 위해 캐스퍼 와인버거 미 국방부 장관과 그레그 전 대사를 한국에 파견한 인물은?
102 이익은 그대로 챙기면서 소비자에게 부담을 주는 방식을 시도해 코즈마케팅에 실패한 기업은?
104 트윈 엘리베이터가 승강기 간의 충돌사고를 방지하기 위해 도입한 기술은 무엇인가?
109 맥스가 본사로 복귀하기 위해 활용한 인물은?
114 신비적 체험에 대해 '삶의 원천에서 깨끗하고 맑은 물을 마신 것 같다' 고 표현한 사람은?
116 메가비조스가 도시를 빼앗고, 나중에 생길 반란을 피하기 위해 부순 것은?
122 매년 해수온이 올라가는 현상으로 인해, 자리돔의 서식지가 어디까지 확장되었는가?
125 나치 독일이 지식인이나 귀족 등 65000여 명의 폴란드인을 잡아들이기 위해 준비한 것은?
127 치킨 누들 한 캔을 생산하기 위해 참여하는 회사의 수는?
131 연병환이 영국인 친구의 소개를 통해 구한 중국의 직장은 어디인가?
133 인간의 의식 중, 사이코사이버네틱스를 통해 통제할 수 있는 것은?
158 수령자가 매매계약을 파기하기 위해서 지불해야하는 것은?
172 고려시대 때 마을의 걱정을 없애기 위해 만든 석불은 무엇으로 만들어졌나요?
177 정치하는 엄마들이 피해자들이 정상적인 일상생활을 할 수 있도록 돕기 위해 진행했던 서비스는?
190 중국은 무엇을 통해 리덩후이의 총통 선거를 방해하려 했는가?
204 헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?
219 일반고 슬럼화 문제를 해결하기 위해 교육청에서 내세운 정책 슬로건은?
236 금고를 걸어두기 위해 만들어진 것은 무슨 모양인가요?
237 카스파는 어떤 역할을 담당해 베토밴을 도와주었나?
251 무기를 개편하기 위해 플레이어가 가지고 있어야 하는 것은?
265 바실리 박이 안전을 위해 만나러 간 사람은?
266 영토를 넓히기 위해 대외정

KeyboardInterrupt: 